In [62]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
# from surprise import Reader, Dataset, SVD, evaluate
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

from uszipcode import SearchEngine
from  geopy.distance import distance


MAX_DISTANCE = 15

import warnings; warnings.simplefilter('ignore')

In [63]:
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [64]:

#!dir .\Data\*.tsv

In [65]:
folder = 'Data'

apps = pd.read_csv('./'+folder+'/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('./'+folder+'/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('./'+folder+'/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('./'+folder+'/users.tsv' ,delimiter='\t',encoding='utf-8')
test_users = pd.read_csv('./'+folder+'/test_users.tsv', delimiter='\t',encoding='utf-8')


b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'


### EDA

In [66]:
apps.head()


,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [67]:
user_history.head()

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer
2,47,1,Train,3,"Passenger Screener, TSA"
3,72,1,Train,1,"Lecturer, Department of Anthropology"
4,72,1,Train,2,Student Assistant


In [68]:
jobs.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [69]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0


In [70]:
test_users.head()
len(users)

389708

In [71]:
users = users.loc[users.State == 'NY']
len(users)

20590

In [72]:
users = users.sample(frac=0.2, replace=False, random_state=1)
len(users)

4118

#### Subsetting jobs in NY

In [73]:
jobs = jobs.loc[jobs.State == 'NY']
jobs = jobs.sample(frac=0.2, replace=False, random_state=1)
#jobs.to_csv("NYjobs.tsv",  sep='\t',encoding='utf-8')

In [74]:
len(jobs)

10801

In [75]:
import gc
gc.collect()

22

In [76]:

jobs.groupby(['Zip5', 'City']).size().reset_index(name='Count').sort_values('Count', ascending=False).head()

,Zip5,City,Count
1,10001,New York,396
725,10001,New York,261
19,10017,New York,118
743,10017,New York,102
18,10016,New York,90


### Preprocessing

### Preprocessing Description and Requirements

In [77]:
import re

def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [78]:

jobs['Description'] = jobs['Description'].astype(dtype='str').apply(preprocessor)

In [79]:
jobs['Requirements'] = jobs['Requirements'].astype(dtype='str').apply(preprocessor)

### Creating jobs coordinates

In [80]:
search = SearchEngine(simple_zipcode=True)

In [81]:
jobs.Zip5 = jobs.Zip5.fillna(0)
jobs.Zip5 = jobs.Zip5.astype(int)

In [82]:
#lat, "long"
"""coords_1 = c
coords_2 = (34.1, -118.42)

print (round(distance(coords_1, coords_2).miles, 2))"""
search.by_zipcode("2e").lat

In [83]:

def coordinates(zipcode):
    zipcode = search.by_zipcode(zipcode)
    community = zipcode.post_office_city
    return community, "{},{}".format(zipcode.lat, zipcode.lng)


In [84]:
%%timeit
jobs["Community"] = ""
jobs["Coordinates"] = ""
for zipcode in jobs.Zip5.unique():
    community, coordinate = coordinates(zipcode)
    jobs.loc[jobs.Zip5 == zipcode, "Coordinates"] =  str(coordinate)
    jobs.loc[jobs.Zip5 == zipcode, "Community"] =  community

4.43 s ± 138 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
jobs.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
277,"New York, NY",2173
342,"Rochester, NY",306
48,"Buffalo, NY",234
46,"Brooklyn, NY",195
44,"Bronx, NY",174
251,"Melville, NY",149
2,"Albany, NY",107
391,"Syracuse, NY",98
437,"White Plains, NY",85
173,"Hauppauge, NY",81


In [86]:
jobs.groupby(['Community', 'Zip5', 'Coordinates']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Zip5,Coordinates,Count
393,"New York, NY",10001,"40.75,-73.99",661
407,"New York, NY",10017,"40.75,-73.973",221
406,"New York, NY",10016,"40.74,-73.97",167
362,"Melville, NY",11747,"40.79,-73.4",149
423,"New York, NY",10036,"40.76,-73.99",138
408,"New York, NY",10018,"40.76,-73.99",109
412,"New York, NY",10022,"40.76,-73.97",102
397,"New York, NY",10005,"40.705,-74.005",82
277,"Hauppauge, NY",11788,"40.82,-73.21",81
409,"New York, NY",10019,"40.77,-73.99",78


### creating user coordinates

In [87]:

def creating_coordinates(df, column):
    df["Community"] = ""
    df["Coordinates"] = ""
    for zipcode in df[column].unique():
        community, coordinate = coordinates(zipcode)
        df.loc[df[column] == zipcode, "Coordinates"] =  str(coordinate)
        df.loc[df[column] == zipcode, "Community"] =  community
    return df
users = creating_coordinates(users, 'ZipCode')

In [88]:
users.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
49,"Brooklyn, NY",544
284,"New York, NY",446
46,"Bronx, NY",405
354,"Rochester, NY",200
397,"Staten Island, NY",90
208,"Jamaica, NY",88
458,"Yonkers, NY",62
51,"Buffalo, NY",61
138,"Flushing, NY",38
11,"Astoria, NY",36


### Distance Matrix 

In [89]:
users.head(2)

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates
200098,248008,4,Train,New York,NY,US,10003,None,Electrical Engineering,NaN,2,15.0,Yes,No,0,"New York, NY","40.73,-73.99"
286529,1184529,5,Train,New York,NY,US,10035,High School,Not Applicable,2005-01-01 00:00:00,3,7.0,NaN,No,0,"New York, NY","40.8,-73.93"


In [90]:
jobs.head(2)

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate,Community,Coordinates
442049,13461,3,Social Worker,healthcare partners ipa andhealthcare partners...,requirements msw and nys license clinical soci...,Garden City,NY,US,11530,2012-04-13 16:28:34.767,2012-05-12 23:59:59,"Garden City, NY","40.73,-73.64"
510877,564493,3,Sovereign Analyst,provide timely and relevant research to pm s ...,nan,New York,NY,US,0,2012-04-12 14:16:54.76,2012-05-11 23:59:59,None,"None,None"


In [91]:
apps.head(2)

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009


In [92]:
user_history.head(2)

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer


### Subset most applied jobs by title

### Looking similar jobs

In [93]:
current_app_intercep = np.intersect1d(users.UserID.unique(), apps.UserID.unique())
historical_app_intercep = np.intersect1d(users.UserID.unique(), user_history.UserID.unique())
total_users = users.UserID.unique()
with_info =  set(current_app_intercep.tolist() + historical_app_intercep.tolist())

current_app = 1-len(current_app_intercep )/len(total_users )
historical_app = 1-len(historical_app_intercep)/ len(total_users )
cold_star_p = len(with_info)/len(total_users )

print(current_app, historical_app, cold_star_p)


0.1733851384167071 0.03302574065080133 0.994900437105391


In [94]:
#distance_jobs_df = pd.DataFrame(columns=jobs.JobID.unique(), index = users.index)

### Looks for most applied jobs id

### Distance matrix

In [95]:


#distance_jobs_df = pd.DataFrame(columns=jobs.JobID.unique(), index = users.index)

In [96]:
#distance_jobs_df.to_csv()

### Recomender top k liked

In [97]:
def user_exist(user):
    if len(users.loc[users['UserID'] == user]) == 0:
        return False
    return True

def has_coordinates(user):
    if len(users.loc[users['UserID'] == user, "Coordinates"]) == 0:
        return False
    return True
    

In [98]:
unique = 0.3 #update name
top = 20

def ranking_by_popularity(top):
    
    popular_jobs = user_history.groupby(
    ['JobTitle']).size().reset_index(
    name='Count').sort_values('Count', ascending=False)
    ranking =  dict()
    top_i = 0
    
    while True:
        job_title = popular_jobs['JobTitle'].iloc[top_i]
        jobs_list = jobs.loc[jobs['Title'] == job_title, ['JobID']]['JobID'].unique().tolist()

        if len(jobs_list) > 1:
            ranking[job_title] = jobs_list

        if len(ranking) == top:
            break

        top_i +=1
    return ranking

ranking_popular = ranking_by_popularity(top)

In [99]:

def recommender_popular_jobs(user, unique, top):
    
    recommended_popular_jobs = dict()
    c1 = users.loc[users['UserID'] == user, 'Coordinates']
    
    if user_exist(user) and has_coordinates(user):
        for title, jobs_list in ranking_popular.items():
            
            distances = dict()
            
            for job in jobs_list:
                
                c2 = jobs.loc[jobs['JobID'] == job, 'Coordinates']
                if c2.iloc[0].split(',')[0] == 'None':
                    continue
                distances[job] = round(distance(c1, c2).miles, 2)
            
            distances = sorted(distances.items(), key=lambda kv: kv[1])
            closest = distances[0]
            if closest[1] >= MAX_DISTANCE:
                continue
            recommended_popular_jobs[title] = (closest[0], closest[1])
            
    else:
        for title, jobs_list in ranking_popular.items():
            recommended_popular_jobs[title] = jobs_list[0]
            
    return recommended_popular_jobs

recommender_popular_jobs(98, unique, top)

{'Customer Service Representative': 1021544,
 'Administrative Assistant': 330119,
 'Sales Associate': 542022,
 'Assistant Manager': 809332,
 'Office Manager': 138054,
 'Manager': 5415,
 'Receptionist': 776478,
 'Customer Service': 473162,
 'Store Manager': 191995,
 'General Manager': 258044,
 'Sales Representative': 510053,
 'Medical Assistant': 447257,
 'Customer Service Rep': 437295,
 'Project Manager': 264489,
 'Office Assistant': 612148,
 'Intern': 923688,
 'Operations Manager': 32863,
 'Account Executive': 587531,
 'Driver': 40865,
 'Account Manager': 44438}

### Content Based title x profile



In [454]:
jobs['profile'] = jobs['Title'].astype(str)  +  '. ' + jobs['Description'].astype(str) + '. ' + jobs['Requirements'].astype(str)

In [426]:
%%timeit

users['profile'] = users['DegreeType'].astype(str) + " " + users['Major'].astype(str) 
#+ str(p.number_to_words(users["TotalYearsExperience"])) + " Years of Experience " + 


1.89 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### Garbage words

In [427]:
users['profile'] = users['profile'].str.replace('None.', '.')
users['profile'] = users['profile'].str.replace('Not Applicable', '.')
users['profile'] = users['profile'].str.replace(' nan', ' ') 

In [279]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

import pickle

In [470]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        self.epoch += 1
        

In [477]:
%%time
def similarities(top, model_name = "jobs_doc2vec_model_2",
                 mapping_name = "jobID_mapping_2.p", max_epochs = 100,
                 alpha = 0.025):
    
    document = list()
    jobID_mapping = dict()
        
    for i, token in enumerate(jobs['profile']):
        value = jobs.iloc[i]["JobID"]
        tokens = TaggedDocument(simple_preprocess(token), [i])
        document.append(tokens)
        jobID_mapping[i] = value

    epoch_logger = EpochLogger()
    model = Doc2Vec(size = 50, alpha=alpha, 
                    min_alpha=0.00025, min_count=1,
                    callbacks=[epoch_logger], dm =1, workers=4
                   )
    
    model.build_vocab(document)
    
    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(document, 
                    total_examples=model.corpus_count, 
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    # create a dictionary
    pickle.dump(model, open(model_name, "wb")) 
    pickle.dump(jobID_mapping, open(mapping_name, "wb")) 

# -------------------------------------------------------------
# Load the dictionary back from the pickle file.
    return model, jobID_mapping

similarities(top = 100)
jobID_mapping = pickle.load(open("jobID_mapping.p", "rb"))
model  = pickle.load(open("jobs_doc2vec_model", "rb"))

iteration 0
Epoch #0 start
Epoch #1 start
Epoch #2 start
Epoch #3 start
Epoch #4 start
iteration 1
Epoch #5 start
Epoch #6 start
Epoch #7 start
Epoch #8 start
Epoch #9 start
iteration 2
Epoch #10 start
Epoch #11 start
Epoch #12 start
Epoch #13 start
Epoch #14 start
iteration 3
Epoch #15 start
Epoch #16 start
Epoch #17 start
Epoch #18 start
Epoch #19 start
iteration 4
Epoch #20 start
Epoch #21 start
Epoch #22 start
Epoch #23 start
Epoch #24 start
iteration 5
Epoch #25 start
Epoch #26 start
Epoch #27 start
Epoch #28 start
Epoch #29 start
iteration 6
Epoch #30 start
Epoch #31 start
Epoch #32 start
Epoch #33 start
Epoch #34 start
iteration 7
Epoch #35 start
Epoch #36 start
Epoch #37 start
Epoch #38 start
Epoch #39 start
iteration 8
Epoch #40 start
Epoch #41 start
Epoch #42 start
Epoch #43 start
Epoch #44 start
iteration 9
Epoch #45 start
Epoch #46 start
Epoch #47 start
Epoch #48 start
Epoch #49 start
iteration 10
Epoch #50 start
Epoch #51 start
Epoch #52 start
Epoch #53 start
Epoch #54 sta

Epoch #424 start
iteration 85
Epoch #425 start
Epoch #426 start
Epoch #427 start
Epoch #428 start
Epoch #429 start
iteration 86
Epoch #430 start
Epoch #431 start
Epoch #432 start
Epoch #433 start
Epoch #434 start
iteration 87
Epoch #435 start
Epoch #436 start
Epoch #437 start
Epoch #438 start
Epoch #439 start
iteration 88
Epoch #440 start
Epoch #441 start
Epoch #442 start
Epoch #443 start
Epoch #444 start
iteration 89
Epoch #445 start
Epoch #446 start
Epoch #447 start
Epoch #448 start
Epoch #449 start
iteration 90
Epoch #450 start
Epoch #451 start
Epoch #452 start
Epoch #453 start
Epoch #454 start
iteration 91
Epoch #455 start
Epoch #456 start
Epoch #457 start
Epoch #458 start
Epoch #459 start
iteration 92
Epoch #460 start
Epoch #461 start
Epoch #462 start
Epoch #463 start
Epoch #464 start
iteration 93
Epoch #465 start
Epoch #466 start
Epoch #467 start
Epoch #468 start
Epoch #469 start
iteration 94
Epoch #470 start
Epoch #471 start
Epoch #472 start
Epoch #473 start
Epoch #474 start
ite

In [638]:

def content_based_recommender(user_id, jobID_mapping = jobID_mapping, model =  model, top = 10):
    #As infer_vector produce stochastics result I made a for to save the best list
    user_profile = np.array(users.loc[users['UserID'] == user_id, 'profile'])[0]
    historical_apps = user_history.loc[user_history.UserID == user_id, 'JobTitle']
    for application in historical_apps:
        user_profile += ". " + str(application)
    user_profile = simple_preprocess(user_profile)
    
    best = 0
    top10=pd.DataFrame(index = range(top), columns = ['JobID', 'Title', 'Description', 'Requirements'])

    for i in range (100):
        inferred_vector = model.infer_vector(user_profile)
        sims = model.docvecs.most_similar([inferred_vector])
        sum_results = 0
        
        for i in range(top):
            sum_results+=sims[0][1]
        if sum_results > best:
            best = sum_results
            for i in range(top):
                recomendation = jobID_mapping[sims[i][0]]
                top10.iloc[i] = np.array(jobs.loc[jobs['JobID'] == recomendation][['JobID', 'Title', 
                                                                       'Description', 'Requirements']])[0]
    
    return top10, sims


In [644]:
t, s = content_based_recommender(248008, top=5)
t

,JobID,Title,Description,Requirements
0,1080071,ADMINISTRATIVE/CLERICAL,colating,nan
1,848914,GENERAL LABOR,none,nan
2,1077716,Sales Consultant,open positions for career minded people in the...,please refer to the job description for the qu...
3,546256,Sales Consultant,open positions for career minded people are yo...,please refer to the job description for the qu...
4,1077719,Sales Consultant,open positions for career minded people in our...,please refer to the job description for the qu...


In [603]:
s[0][1]

0.5770403742790222

#### User to User
#### Distance Recommender



### Testing